### TireTree-based Keyword Search

In [1]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end = False

In [2]:
class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    def insert(self, word):
        node = self.root
        
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end = True
        
    def find(self, prefix):
        node = self.root
        
        for char in prefix:
            node = node.children.get(char)
            if not node and char in self.root.children:
                node = self.root.children.get(char)
            elif not node:
                node = self.root
            if node.is_end:
                return True
           

        return False

In [3]:
keywords = ["台積電", "晶圓", "電子", "電路", "半導體", "韌體", "台積"]

In [4]:
with open('./news_titles.txt', 'r') as f:
    titles = f.readlines()

In [5]:
def Unicode():
    val = random.randint(0x4e00, 0x9fbf)
    return chr(val)

In [6]:
import random 
import string 
letters = string.ascii_letters
random_keywords = []
for i in range(1000):
    random_keywords.append(''.join(Unicode() for i in range(3)))

In [7]:
random_keywords[:10]

['韶牠囝', '渙綃化', '曉岉構', '怽筵彧', '鳂圛鸭', '膫滁甤', '闃韕躄', '诛獹溁', '宖峧貛', '畵椿鑬']

In [8]:
import time
num_keywords = []
trie_costs = []
bf_costs = []
for i in range(1,1002,250):
    tmp_keywords = random_keywords[0:i] + keywords
    tmp_trie_costs = []
    tmp_bf_costs = []
    num_titles = []
    num_keywords.append(len(tmp_keywords))
    for i in range(1, 251, 50):
        tmp_titles = titles * i
        num_titles.append(len(tmp_titles))
        
        # trie tree
        start_time = time.time()
        trie_tree = Trie()
        is_matches = []
        for keyword in tmp_keywords:
            trie_tree.insert(keyword)
        for title in tmp_titles:
            is_matches.append(trie_tree.find(title))
        tmp_trie_costs.append(time.time() - start_time)
        
        # brute force
        start_time = time.time()
        bf_results = []
        for title in tmp_titles:
            for keyword in tmp_keywords:
                if keyword in title:
                    bf_results.append(title)
                    break
        tmp_bf_costs.append(time.time() - start_time)
    trie_costs.append(tmp_trie_costs)
    bf_costs.append(tmp_bf_costs)

In [9]:
import pandas as pd
trie_df = pd.DataFrame(data=trie_costs,columns=num_titles,index=num_keywords)
trie_df

,340,17340,34340,51340,68340
8,0.001697,0.075026,0.144685,0.217373,0.286705
258,0.002223,0.075478,0.149278,0.221971,0.294353
508,0.002812,0.074785,0.149163,0.220198,0.295152
758,0.003238,0.073835,0.145732,0.217291,0.292947
1007,0.019242,0.077526,0.150745,0.223519,0.301813


In [10]:
bf_df = pd.DataFrame(data=bf_costs,columns=num_titles,index=num_keywords)
bf_df

,340,17340,34340,51340,68340
8,0.000221,0.010913,0.021603,0.032279,0.043170
258,0.005708,0.292650,0.581531,0.868970,1.151677
508,0.011276,0.572185,1.135246,1.696363,2.253173
758,0.016731,0.852622,1.688779,2.524114,3.352198
1007,0.022189,1.129941,2.240231,3.349073,4.452981


In [8]:
!pip install tabulate

You should consider upgrading via the '/home/ubuntu/anaconda3/bin/python -m pip install --upgrade pip' command.


In [11]:
print(trie_df.to_markdown())

|      |        340 |     17340 |    34340 |    51340 |    68340 |
|-----:|-----------:|----------:|---------:|---------:|---------:|
|    8 | 0.0016973  | 0.075026  | 0.144685 | 0.217373 | 0.286705 |
|  258 | 0.00222325 | 0.0754781 | 0.149278 | 0.221971 | 0.294353 |
|  508 | 0.00281191 | 0.0747848 | 0.149163 | 0.220198 | 0.295152 |
|  758 | 0.00323772 | 0.0738354 | 0.145732 | 0.217291 | 0.292947 |
| 1007 | 0.0192423  | 0.0775256 | 0.150745 | 0.223519 | 0.301813 |


In [12]:
print(bf_df.to_markdown())

|      |         340 |     17340 |     34340 |     51340 |     68340 |
|-----:|------------:|----------:|----------:|----------:|----------:|
|    8 | 0.000220537 | 0.0109131 | 0.0216031 | 0.0322795 | 0.0431697 |
|  258 | 0.00570846  | 0.29265   | 0.581531  | 0.86897   | 1.15168   |
|  508 | 0.0112755   | 0.572185  | 1.13525   | 1.69636   | 2.25317   |
|  758 | 0.016731    | 0.852622  | 1.68878   | 2.52411   | 3.3522    |
| 1007 | 0.0221889   | 1.12994   | 2.24023   | 3.34907   | 4.45298   |


In [11]:
trie_tree = Trie()
for keyword in keywords:
    trie_tree.insert(keyword)

In [12]:
len(keywords)

7

In [13]:
%%time
is_matches = []
for title in titles:
    is_matches.append(trie_tree.find(title))

CPU times: user 1.63 ms, sys: 3 µs, total: 1.63 ms
Wall time: 1.63 ms


In [14]:
from itertools import compress
results = list(compress(titles, is_matches))

In [15]:
%%time
bf_results = []
for title in titles:
    for keyword in keywords:
        if keyword in title:
            bf_results.append(title)
            break

CPU times: user 214 µs, sys: 0 ns, total: 214 µs
Wall time: 216 µs
